## Notebook to perform the vertical transformation from z-levels to sigma coordinates for the zonal velocity


The vertical transformation is performed using linear interpolation similarly as in model2roms

In [5]:
# Import packages
import numpy as np
import xarray as xr


In [9]:
# import data

# Import the data that needs to be vertically transformed (MODEL grid)
data_in = xr.open_dataset('horizontally_regridded.nc')

# Import ROMS grid
grid = xr.open_dataset('/Users/iriskeizer/Documents/ROMS/data/grid/NorthSea4_smooth01_sponge_nudg.nc')

data = data_in.drop(['so', 'zos', 'thetao'])



In [1]:
903/60

15.05

In [10]:
# Create the dataset grdMODEL

grdMODEL = data.copy()

grdMODEL = grdMODEL.drop(['uo', 'vo'])

grdMODEL['lon'] = data.lon
grdMODEL['lat'] = data.lat
grdMODEL['h'] = data.depth
grdMODEL['nlevels'] = grdMODEL.h.size

grdMODEL['fillval'] = -32767
grdMODEL['hc'] = None

# Create grid for ESMF interpolation, probably not needed for VT

grdMODEL['z_r'] = - grdMODEL.h

grdMODEL['grdType'] = 'regular'
grdMODEL['lonName'] = 'longitude'
grdMODEL['latName'] = 'latitude'
grdMODEL['depthName'] = 'depth'


grdMODEL['Lp'] = len(grdMODEL.lat[1,:])
grdMODEL['Mp'] = len(grdMODEL.lat[:,1])

grdMODEL['L'] = grdMODEL.Lp -1
grdMODEL['M'] = grdMODEL.Mp -1


In [11]:
# Create the dataset grdROMS

# Copy the roms grid
grdROMS = grid.copy()

# Drop unnecessary variables
grdROMS = grdROMS.drop(['tracer_NudgeCoef', 'diff_factor', 'visc_factor', 'hraw', 'f', 'spherical'])



# Add below variables to grdROMS
grdROMS['write_clim'] = True
grdROMS['write_bry'] = True
grdROMS['write_init'] = True
grdROMS['write_stations'] = False
grdROMS['lonname'] = 'lon_rho'
grdROMS['latname'] = 'lat_rho'
grdROMS['inittime'] = 0                    # Set initTime to 1 if you dont want the first time-step to be the initial field (no ubar and vbar if time=0)
grdROMS['ocean_time'] = 0
grdROMS['NT'] = 2
grdROMS['tracer'] = grdROMS.NT
grdROMS['time'] = 0                      
grdROMS['reftime'] = 0
grdROMS['grdtype'] = 'regular'

grdROMS['masked_h'] = grdROMS.h.where(grdROMS.h > 0, grdROMS.h, grdROMS.h.max())
grdROMS['hmin'] = grdROMS.masked_h.min()

grdROMS['vtransform'] = 2
grdROMS['vstretching'] = 4

grdROMS['nlevels'] = grdROMS.s_rho.size

grdROMS['zeta'] = (('eta_rho', 'xi_rho'), np.zeros(grdROMS.h.shape))

grdROMS['invpm'] = 1.0 / grdROMS.pm
grdROMS['invpn'] = 1.0 / grdROMS.pn

grdROMS['Lp'] = grdROMS.lat_rho[1,:].size     
grdROMS['Mp'] = grdROMS.lat_rho[:,1].size     

grdROMS['fillval'] = -9.99e33

grdROMS['eta_rho_'] = grdROMS.Mp
grdROMS['eta_u_'] = grdROMS.Mp
grdROMS['eta_v_'] = grdROMS.Mp - 1
grdROMS['eta_psi_'] = grdROMS.Mp - 1


grdROMS['xi_rho_'] = grdROMS.Lp
grdROMS['xi_u_'] = grdROMS.Lp - 1
grdROMS['xi_v_'] = grdROMS.Lp
grdROMS['xi_psi_'] = grdROMS.Lp - 1





# Obtain s_rho

c1 = 1.0
c2 = 2.0
p5 = 0.5

lev = np.arange(1, int(grdROMS.nlevels) + 1, 1)
ds = 1.0 / int(grdROMS.nlevels)


grdROMS['s_rho_'] = - c1 + (lev - p5) * ds


# Obtain s_w

lev = np.arange(0, int(grdROMS.nlevels), 1)
ds = 1.0 / (int(grdROMS.nlevels) - 1)


grdROMS['s_w_'] = - c1 + (lev - p5) * ds




# Obtain Cs_r

if (grdROMS.theta_s > 0):
    Csur = (c1 - np.cosh(grdROMS.theta_s * grdROMS.s_rho)) / (np.cosh(grdROMS.theta_s) - c1)
        
else:
    Csur = -grdROMS.s_rho**2
        
if (grdROMS.theta_b > 0):
    Cbot = (np.exp(grdROMS.theta_b * Csur) - c1 ) / (c1 - np.exp(-grdROMS.theta_b))
    grdROMS['Cs_r'] = Cbot
else:
    grdROMS['Cs_r'] = Csur     


    
# Obtain Cs_w

if (grdROMS.theta_s > 0):
    Csur = (c1 - np.cosh(grdROMS.theta_s * grdROMS.s_w)) / (np.cosh(grdROMS.theta_s) - c1)
        
else:
    Csur = -grdROMS.s_w**2
        
if (grdROMS.theta_b > 0):
    Cbot = (np.exp(grdROMS.theta_b * Csur) - c1 ) / (c1 - np.exp(-grdROMS.theta_b))
    grdROMS['Cs_w'] = Cbot
else:
    grdROMS['Cs_w'] = Csur     


    

# Obtain z_r

z0 = (grdROMS.hc * grdROMS.s_rho + grdROMS.h * grdROMS.Cs_r) / (grdROMS.hc + grdROMS.h)
grdROMS['z_r'] = grdROMS.zeta + (grdROMS.zeta + grdROMS.h) * z0

    

# Obtain z_w

z0 = (grdROMS.hc * grdROMS.s_w + grdROMS.h * grdROMS.Cs_w) / (grdROMS.hc + grdROMS.h)
grdROMS['z_w'] = grdROMS.zeta + (grdROMS.zeta + grdROMS.h) * z0



# Also ESMF grid is added but probably not needed for VT



grdROMS['L'] = grdROMS.Lp -1
grdROMS['M'] = grdROMS.Mp -1


Udata, Vdata, UBARdata, VBARdata = vertical_interpolation(myvar, u, v, confM2R.grdROMS,
                                                                                  confM2R.grdMODEL)

In [13]:
# Indices of output data, however my output data includes Time
index_time = data.time.size


outINDEX_U = (int(grdROMS.nlevels), int(grdROMS.eta_u_), int(grdROMS.xi_u_))
outINDEX_UBAR = (int(grdROMS.eta_u_), int(grdROMS.xi_u_))
outINDEX_V = (int(grdROMS.nlevels), int(grdROMS.eta_v_), int(grdROMS.xi_v_))
outINDEX_VBAR = (int(grdROMS.eta_v_), int(grdROMS.xi_v_))


outdataU = np.empty((outINDEX_U))
outdataUBAR = np.empty((outINDEX_UBAR))
outdataV = np.empty((outINDEX_V))
outdataVBAR = np.empty((outINDEX_VBAR))


outdataU = interp.interpolation.dovertinter(np.asarray(outdataU, order='F'),
                                                    np.asarray(array1, order='F'),
                                                    np.asarray(grdROMS.h, order='F'),
                                                    np.asarray(grdROMS.z_r, order='F'),
                                                    np.asarray(grdMODEL.z_r, order='F'),
                                                    int(grdROMS.nlevels),
                                                    int(grdMODEL.nlevels),
                                                    int(grdROMS.xi_u),
                                                    int(grdROMS.eta_u),
                                                    int(grdROMS.xi_rho),
                                                    int(grdROMS.eta_rho))
                                                    
                                                    
                                                    
doVertInter(outdat,dat,bathymetry,zr,zs,Nroms,Nsoda,II,JJ,xi_rho,eta_rho)

In [ ]:
# So, the function interp.interpolation.dovertinterp is called and necessary is:
outdat = outdataU
dat = data
bathymetry = grdROMS.h
zr = grdROMS.z_r
zs = grdMODEL.z_r
Nroms = grdROMS.nlevels
Ndata = grdMODEL.nlevels
II = int(grdROMS.xi_u_)
JJ = int(grdROMS.eta_u_)
xi_rho = int(grdROMS.xi_rho_)
eta_rho = int(grdROMS.eta_rho_)
fill=-10000

# Transpose dimensions
zr = zr.transpose('s_rho', 'eta_rho', 'xi_rho')

